US Consumer Behaviour 

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
pce = pd.read_csv("../data/PERSONAL_EXPENDITURE.csv")
save = pd.read_csv("../data/SAVING_RATE.csv")
credit = pd.read_csv("../data/CREDIT_OWNED.csv")

In [32]:
for df in [credit, save, pce]:
    df.rename(columns={'observation_date': 'DATE'}, inplace=True)
    df['DATE'] = pd.to_datetime(df['DATE'])

In [33]:
master_df = pce.merge(save, on='DATE', how='inner') \
                  .merge(credit, on='DATE', how='inner')

In [34]:
master_df.rename(columns={
    'PCEC96': 'Expenditure_Billions',
    'PSAVERT': 'Saving_Rate_Pct',
    'TOTALSL': 'Credit_Owned_Billions'
}, inplace=True)

In [35]:
master_df.sort_values('DATE', inplace=True)

In [36]:
master_df['Credit_to_Spend_Ratio'] = master_df['Credit_Owned_Billions'] / master_df['Expenditure_Billions']

master_df['Spend_YoY_Growth'] = master_df['Expenditure_Billions'].pct_change(periods=12) * 100
master_df['Credit_YoY_Growth'] = master_df['Credit_Owned_Billions'].pct_change(periods=12) * 100


def assign_regime(date):
    if date < pd.to_datetime('2020-03-01'):
        return '1_Pre_Covid'
    elif date < pd.to_datetime('2021-06-01'):
        return '2_Covid_Stimulus'
    else:
        return '3_Post_Inflation_Shock'

master_df['Regime'] = master_df['DATE'].apply(assign_regime)


master_df.dropna(inplace=True)


print(master_df.head())

         DATE  Expenditure_Billions  Saving_Rate_Pct  Credit_Owned_Billions  \
12 2008-01-01               11333.2              2.6             2619427.65   
13 2008-02-01               11293.9              3.0             2634496.42   
14 2008-03-01               11322.1              2.9             2645603.64   
15 2008-04-01               11340.5              2.4             2654243.23   
16 2008-05-01               11361.6              6.8             2660193.15   

    Credit_to_Spend_Ratio  Spend_YoY_Growth  Credit_YoY_Growth       Regime  
12             231.128688          1.361238           6.569798  1_Pre_Covid  
13             233.267199          1.035050           6.657618  1_Pre_Covid  
14             233.667221          1.174189           6.487213  1_Pre_Covid  
15             234.049930          1.240905           6.436682  1_Pre_Covid  
16             234.138955          1.280086           5.983113  1_Pre_Covid  
